In [1]:
#Version one
import requests
from lxml import html
import config 

session_request = requests.session()

#print(config.username)

login_url = "https://abacus.co.ke/auth/login"
result = session_request.get(login_url)

tree = html.fromstring(result.text)
authenticating_token = list(set(tree.xpath("//input[@name='_token']/@value")))[0]
#create payload
payload = {'username':config.username,
           'password':config.password,
           '_token':authenticating_token   }

result = session_request.post(
    login_url,
    data = payload,
    headers = dict(referer = login_url)   
)

In [2]:
url = "https://abacus.co.ke/live/company/SCOM/charts"
result = session_request.get(
    url,
    headers = dict(referer=url)
)
#print(result.content)

import json, re, csv
from bs4 import BeautifulSoup
soup = BeautifulSoup(result.content, 'html.parser')
script = soup.find('script', text=re.compile('Performance = ({.*?});'))
json_text = re.search(r'Performance = ({.*?});',script.string, flags=re.DOTALL |re.MULTILINE).group(1)
data = json.loads(json_text)
#print(data)
stock_details = data['data']
stock_data = open('./path/to/output/folder/filename.csv', 'w')

# create the csv writer object

csvwriter = csv.writer(stock_data)

count = 0

for stock in stock_details:
    if count == 0:
        header = stock.keys()
        csvwriter.writerow(header)
        count += 1

    csvwriter.writerow(stock.values())

stock_data.close()